# Example 1 - Square Ice Shelf

This notebook models a very simple Square Ice Shelf and is intended to provide a simple example of building an ISSM model using pyISSM.


In [ ]:
import os
import sys
sys.path.append(os.path.expanduser('~/pyISSM/src'))
import pyissm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create an empty model
md = pyissm.model.Model()

---
## 1. Model Mesh

### Build the mesh

In [ ]:
# Create a mesh using the triangle meshing tool
md = pyissm.model.mesh.triangle(md,
                                domain_name = os.path.expanduser('~/pyISSM/tutorials/assets/Exp/square_ice_shelf/DomainOutline.exp'),
                                resolution = 100000)

# Inspect the created mesh
md.mesh

### Visualize the mesh

In [ ]:
# Process the mesh for plotting
mesh, meshX, meshY, meshElements, is3d = pyissm.model.mesh.process_mesh(md)

# Plot the mesh with customised options
fig, ax = pyissm.plot.plot_mesh2d(mesh,
                                  color = 'blue',
                                  linewidth = 0.5,
                                  show_nodes = True,
                                  node_kwargs = {'s': 20,
                                                 'color': 'red',
                                                 'alpha': 0.5})
# We can interact with the plot using matplotlib functions
ax.set_xlabel('X Coordinate (m)')
ax.set_ylabel('Y Coordinate (m)')
ax.set_title('Square Ice Shelf Mesh')

---
## 2. Model Mask

### Define the mask
The `md.mask.ice_levelset` and `md.mask.ocean_levelset` fields interact to define where there is grounded ice, floating ice, ice-free regions, and open ocean within the model domain. In this example, the entire domain is floating ice. We can use the `set_mask()` function to efficiently define the `md.mask` fields.

In [ ]:
# Define the mask: all ice is floating
md = pyissm.model.param.set_mask(md,
                                 floating_ice_name = 'all',
                                 grounded_ice_name = None)

# Inspect the mask
md.mask

### Visualise the mask

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
pyissm.plot.plot_model_field(md,
                             md.mask.ice_levelset,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax1)
ax1.set_title('md.mask.ice_levelset')

pyissm.plot.plot_model_field(md,
                             md.mask.ocean_levelset,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax2)
ax2.set_title('md.mask.ocean_levelset')

pyissm.plot.plot_model_elements(md,
                                ice_levelset = md.mask.ice_levelset,
                                ocean_levelset = md.mask.ocean_levelset,
                                type = 'floating_ice_elements',
                                show_mesh = True,
                                ax = ax3)
ax3.set_title('Floating ice')
plt.tight_layout()

---
## 3. Parameterise the model

<div class="alert alert-block alert-warning">
<b>pyissm.model.param.parameterize():</b> The following blocks of code used to parameterise the model can be embedded into a separate python file, and parsed to the model using:

<pre><code>md = pyissm.model.param.parameterize('PARAM_FILE.py')</code></pre>
 
This functions **exactly** the same as running the code directly, but helps to keep your main model execution scripts clean. We embed the code directly here for transparency.
</div>

### Define model geometry

In [ ]:
# Define constants
hmin = 300
hmax = 1000
ymin = np.nanmin(md.mesh.y)
ymax = np.nanmax(md.mesh.y)

# Assign geometry to the model
md.geometry.thickness = hmax + (hmin - hmax) * (md.mesh.y - ymin) / (ymax - ymin)
md.geometry.base = - md.materials.rho_ice / md.materials.rho_water * md.geometry.thickness
md.geometry.surface = md.geometry.base + md.geometry.thickness

# Inspect the geometry
md.geometry

In [ ]:
# Visualise the model geometry
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
pyissm.plot.plot_model_field(md,
                             md.geometry.thickness,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax1)
ax1.set_title('Ice Thickness (m)')

pyissm.plot.plot_model_field(md,
                             md.geometry.base,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax2)
ax2.set_title('Ice Base Elevation (m)')

pyissm.plot.plot_model_field(md,
                             md.geometry.surface,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax3)
ax3.set_title('Ice Surface Elevation (m)')
plt.tight_layout()

### Define friction

In [ ]:
# Define friction parameters
md.friction.coefficient = np.where(md.mask.ocean_levelset < 0., 0, 200)
md.friction.p = np.ones((md.mesh.numberofelements))
md.friction.q = np.ones((md.mesh.numberofelements))

# Inspect friction parameters
md.friction

In [ ]:
# Visualise the friction fields
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
pyissm.plot.plot_model_field(md,
                             md.friction.coefficient,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax1)
ax1.set_title('Friction Coefficient')

pyissm.plot.plot_model_field(md,
                             md.friction.p,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax2)
ax2.set_title('Friction Exponent p')

pyissm.plot.plot_model_field(md,
                             md.friction.q,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax3)
ax3.set_title('Friction Exponent q')
plt.tight_layout()

### Define initial velocity

In [ ]:
# Define initial velocities
md.initialization.vx = np.zeros((md.mesh.numberofvertices))
md.initialization.vy = np.zeros((md.mesh.numberofvertices))
md.initialization.vz = np.zeros((md.mesh.numberofvertices))
md.initialization.vel = np.zeros((md.mesh.numberofvertices))

# Inspect initialization fields
md.initialization

In [ ]:
# Visualise the initial velocities
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 6))
pyissm.plot.plot_model_field(md,
                             md.initialization.vx,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax1)
ax1.set_title('md.initialization.vx (m/a)')
pyissm.plot.plot_model_field(md,
                             md.initialization.vy,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax2)
ax2.set_title('md.initialization.vy (m/a)')
pyissm.plot.plot_model_field(md,
                             md.initialization.vz,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax3)
ax3.set_title('md.initialization.vz (m/a)')
pyissm.plot.plot_model_field(md,
                             md.initialization.vel,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax4)
ax4.set_title('md.initialization.vel (m/a)')
plt.tight_layout()


### Define flow law parameters

In [ ]:
# Define materials parameters
md.materials.rheology_B = pyissm.tools.materials.paterson((273 - 20) * np.ones((md.mesh.numberofvertices)))
md.materials.rheology_n = 3 * np.ones((md.mesh.numberofelements))

# Inspect the materials parameters
md.materials

In [ ]:
# Visualise materials parameters
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
pyissm.plot.plot_model_field(md,
                             md.materials.rheology_B,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax1)
ax1.set_title('md.materials.rheology_B')

pyissm.plot.plot_model_field(md,
                             md.materials.rheology_n,
                             show_cbar = True,
                             show_mesh = True,
                             ax = ax2)
ax2.set_title('md.materials.rheology_n')
plt.tight_layout()

### Define boundary conditions

By default, we set Dirichlet boundary conditions on all inflow nodes and Neumann boundary conditions at the ice-front.

In [ ]:
# Set ice shelf boundary conditions.
md = pyissm.model.bc.set_ice_shelf_bc(md, os.path.expanduser('~/pyISSM/tutorials/assets/Exp/square_ice_shelf/Front.exp'))

# Inspect boundary conditions
# Stress balance boundary conditions are defined by combination of fields in md.stressbalance.spcvx, md.stressbalance.spcvy, md.stressbalance.spcvz.
md.stressbalance

In [ ]:
# Visualise boundary conditions
fig, ax = pyissm.plot.plot_model_bc(md)
ax.set_title('Ice Shelf Boundary Conditions')

---
## 4. Set the flow equation

Here, we use the Shelfy-Stream Approximation (SSA) of the Full-Stokes equation across the whole domain.

In [ ]:
# Use the SSA flow approximation across the whole domain
md = pyissm.model.param.set_flow_equation(md, SSA = 'all')

## 5. Execute the model

Here, we run the model! We use the default `md.cluster` as we can run this small model on a Gadi login node. For larger mdoels, we'd need to change the cluster to use `pyissm.model.classes.cluster.gadi()` and define various parameters here.

Here, the results are loaded back onto `md.results` once the model run has finished.

In [ ]:
## Define an execution directory to house the model results.
md.cluster.executionpath = os.path.expanduser('~/models/')

if os.path.isdir(md.cluster.executionpath):
    print(f'Models will be saved in {md.cluster.executionpath}') 
else:
    raise FileNotFoundError(f'{md.cluster.executionpath} is not a directory. Ensure this directory exists')

In [ ]:
# Give the model a name
md.miscellaneous.name = 'square_ice_shelf'

# Solve the stress balance
md = pyissm.model.execute.solve(md, 'Stressbalance')

### 6. Visualise the model results

In [ ]:
# Visualise the resultant velocity field
fig, ax = pyissm.plot.plot_model_field(md,
                                       field = md.results.StressbalanceSolution.Vel,
                                       show_cbar = True,
                                       cbar_kwargs={'label': 'Ice Velocity (m/a)'},
                                       show_mesh = True)
ax.set_title('Square Ice Shelf Velocity Field')

In [ ]:
# Inspect the resultant velocity field
md.results.StressbalanceSolution.Vel